In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/in_micro_PERSONS_PerthOnly_2011.csv')

In [3]:
df.rename(columns={
    'AREAENUM':'sample_geog',
    "ABSHID":'serialno'
},inplace=True)

zonecol = 'sample_geog'

In [4]:
df.columns

Index(['serialno', 'sample_geog', 'AGEP', 'INCP', 'INDP', 'LFSP', 'OCCP',
       'RLHP', 'SEXP'],
      dtype='object')

In [5]:
df

,serialno,sample_geog,AGEP,INCP,INDP,LFSP,OCCP,RLHP,SEXP
0,CSF11B00068468,51,25,9,16,1,5,5,1
1,CSF11B00068468,51,36,5,22,3,10,6,2
2,CSF11B00068468,51,30,4,22,3,10,1,1
3,CSF11B00068468,51,29,5,22,3,10,1,2
4,CSF11B00068468,51,25,7,10,1,5,5,1
5,CSF11B00068468,51,25,4,22,2,10,5,2
6,CSF11B00074011,50,29,12,3,1,1,1,1
7,CSF11B00074011,50,29,6,7,1,8,1,2
8,CSF11B00074011,50,17,2,22,2,10,4,1
9,CSF11B00074011,50,17,3,8,1,8,4,1


In [6]:
def produce_one_marginal(col, df):
    gdf = pd.DataFrame(df[[zonecol,col]])
    gdf = gdf.groupby([zonecol,col])[col].count()
    return list(gdf.iteritems())

def merge_one_marginal(resdf, newd, col):
    for ad in newd:
        zoneid = ad[0][0]
        catname = col + ';' + str(ad[0][1])
        val = ad[1]
        resdf.at[zoneid, catname] = val
        
def get_twolevel_cols(resdf):
    cols1 = []
    cols2 = []
    for acol in resdf.columns:
        col1 = acol
        col2 = ''
        if ';' in acol:
            acolsplit = acol.split(';')
            col1 = acolsplit[0]
            col2 = acolsplit[1]
        cols1.append(col1)
        cols2.append(col2)
    resdf.columns = cols1
    resdf.loc[-1] = cols2
    resdf.index = resdf.index + 1
    return resdf.sort_index()

def produce_list_marginals(listofmar, df,times = 1, insertgeog = False):
    resdf = pd.DataFrame()
    for col in listofmar:
        newd = produce_one_marginal(col, df)
        merge_one_marginal(resdf,newd,col)
    resdf = resdf.reset_index().rename({"index":"zone"},axis = 1).fillna(0.0)
    resdf = resdf * times
    resdf['zone'] = resdf['zone']/times
    if insertgeog:
        resdf['sample_geog'] = resdf['zone']
    resdf = get_twolevel_cols(resdf)
    return resdf

resdf = produce_list_marginals(['INCP', 'INDP', 'LFSP', 'OCCP','RLHP', 'SEXP'], df, 2)
resdf

,zone,INCP,INCP,INCP,INCP,INCP,INCP,INCP,INCP,INCP,...,RLHP,RLHP,RLHP,RLHP,RLHP,RLHP,RLHP,RLHP,SEXP,SEXP
0,,1,2,3,4,5,6,7,8,9,...,4,5,6,7,8,9,10,11,1,2
1,49,38,448,462,484,488,628,482,494,488,...,382,364,142,1136,210,316,338,124,3794,3862
2,50,42,566,522,674,600,766,668,644,656,...,480,482,188,1106,220,288,110,96,4664,4944
3,51,40,588,466,624,688,758,612,592,610,...,400,506,188,1230,204,254,194,88,4256,4246
4,52,28,480,368,458,492,594,484,456,462,...,364,408,148,814,184,254,132,68,3424,3532


In [7]:
df.to_csv('./data/person_sample.csv',index=False)
resdf.to_csv('./data/person_marginals.csv',index=False)

In [8]:
# start households data things
hdf = pd.read_csv('./data/in_micro_DWELLINGS_PerthOnly_2011.csv')
geomapdf = pd.read_csv('./data/census2011/geocodemap.csv')

In [9]:
geomap = {
    'Bendigo and Shepparton':20,
    'Cairns and Queensland - Outback':35,
    'North West, Warrnambool and South West':31,
    'Sydney - Baulkham Hills and Hawkesbury, Sydney - Ryde':8,
    'Sydney - Inner West':13,
    'Bunbury, Mandurah, Western Australia - Wheat Belt':48,
    }
for idx,row in geomapdf.iterrows():
    geomap[row['Name']] = int(row['code'])
geomap

{'Bendigo and Shepparton': 20,
 'Cairns and Queensland - Outback': 35,
 'North West, Warrnambool and South West': 31,
 'Sydney - Baulkham Hills and Hawkesbury, Sydney - Ryde': 8,
 'Sydney - Inner West': 13,
 'Bunbury, Mandurah, Western Australia - Wheat Belt': 48,
 'Capital Region, Riverina, Southern Highlands and Shoalhaven': 1,
 'Central Coast': 2,
 'Central West, Hunter Valley excl. Newcastle': 3,
 'Coffs Harbour - Grafton, Mid North Coast, Richmond - Tweed': 4,
 'Far West and Orana, Murray, New England and North West': 5,
 'Illawarra': 6,
 'Newcastle and Lake Macquarie': 7,
 'Sydney - Baulkham Hills and Hawkesbury, Sydney - Ryde ': 8,
 'Sydney Blacktown': 9,
 'Sydney - City and Inner South': 10,
 'Sydney - Eastern Suburbs': 11,
 'Sydney - Inner South West': 12,
 'Sydney - Inner West ': 13,
 'Sydney - North Sydney and Hornsby, Sydney - Northern Beaches': 14,
 'Sydney - Outer South West, Sydney - Sutherland': 15,
 'Sydney - Outer West and Blue Mountains': 16,
 'Sydney - Parramatta': 

In [10]:
# hdf = hdf.replace({'AREAENUM':geomap})
hdf.rename(columns={
    'AREAENUM':'sample_geog',
    "ABSHID":'serialno'
},inplace=True)

In [11]:
hdf.columns

Index(['serialno', 'sample_geog', 'BEDRD', 'MRERD', 'RNTRD', 'STRD', 'VEHRD'], dtype='object')

In [12]:
hresdf = produce_list_marginals(['BEDRD', 'MRERD', 'RNTRD', 'STRD', 'VEHRD'], hdf, 2, True)
hresdf

,zone,BEDRD,BEDRD,BEDRD,BEDRD,BEDRD,BEDRD,BEDRD,BEDRD,MRERD,...,STRD,STRD,VEHRD,VEHRD,VEHRD,VEHRD,VEHRD,VEHRD,VEHRD,sample_geog
0,,0,1,2,3,4,5,6,7,1,...,5,6,0,1,2,3,4,5,6,
1,49,14,162,586,1076,782,164,198,268,10,...,2,268,246,996,1064,282,170,224,268,49
2,50,8,74,330,1344,1492,196,178,100,28,...,2,100,166,1080,1488,486,200,202,100,50
3,51,12,96,344,1344,1146,170,162,158,12,...,0,158,212,1022,1214,402,224,200,158,51
4,52,8,72,298,928,1070,138,152,116,26,...,2,116,142,858,1000,312,188,166,116,52


In [13]:
hdf.to_csv('./data/household_sample.csv',index=False)
hresdf.to_csv('./data/hh_marginals.csv',index=False)